In [1]:
from grandma_stock_valuation import FileLogger, YahooDataLoader, batchValuation, addCashPortfolio, allocatePortfolio
import time

logger = FileLogger()
logPrint = logger.log_pandas


### Refresh data of each instrument

In [2]:
d_instrument = {
    'IVV':'SP500',
    '3073.HK':'Greater China',
    'VPL':'Developed Asia-Pacific',
    'IEV':'Europe',
    'ASEA':'SE Asia',

    'XT':'Global Tech',
    'IXG':'Global Finance',
    'IXJ':'Global Healthcare',
    'KXI':'Global Consumer Staple',
    'RXI':'Global Consumer Discretionary',

    'AAXJ':'Asia ex Japan',
    'EZU':'Eurozone'
}

d_instrument_data = {}
for ticker, name in d_instrument.items():
    logPrint(f"{ticker}: {name}")

    # Refresh data
    yahoo = YahooDataLoader(ticker, verbose=0, printfunc=logPrint)
    df = yahoo.queryEOD(save=True)

    d_instrument_data[ticker] = df
    time.sleep(1)



2022-02-18 14:47:14,484 INFO IVV: SP500
2022-02-18 14:47:14,485 INFO IVV: Existing EOD data file found at __data__\IVV_EOD.csv.gz.
2022-02-18 14:47:14,502 INFO IVV: Existing EOD data file contains 5473 rows over 5473 dates from 2000-05-19 to 2022-02-17.
2022-02-18 14:47:14,611 INFO IVV: Queried EOD data contains 1 rows over 1 dates from 2022-02-17 to 2022-02-17.
2022-02-18 14:47:14,616 INFO IVV: Amended data file contains 5473 rows over 5473 dates from 2000-05-19 to 2022-02-17.
2022-02-18 14:47:15,743 INFO 3073.HK: Greater China
2022-02-18 14:47:15,745 INFO 3073.HK: Existing EOD data file found at __data__\3073.HK_EOD.csv.gz.
2022-02-18 14:47:15,753 INFO 3073.HK: Existing EOD data file contains 2817 rows over 2817 dates from 2010-09-15 to 2022-02-18.
2022-02-18 14:47:15,835 ERROR 3073.HK: Failed to query EOD data!
2022-02-18 14:47:15,836 WARNING 3073.HK: No new data to refresh.
2022-02-18 14:47:16,846 INFO VPL: Developed Asia-Pacific
2022-02-18 14:47:16,848 INFO VPL: Existing EOD data 

### Valuate the instruments

In [4]:
df_valuation_metrics, d_fig = batchValuation(d_instrument_data, verbose=0, printfunc=logPrint)

df_valuation_metrics

,ticker,r2_train,train_years,annualized_return,currenct_price,fair_price,over_value_range,over_value_years
0,IVV,0.970875,9.99726,0.134298,439.100006,410.225532,0.070387,0.524109
1,3073.HK,0.916889,10.00274,0.082725,51.540001,52.423723,-0.016857,-0.139451
2,VPL,0.890417,9.99726,0.067471,75.190002,77.105931,-0.024848,-0.167653
3,IEV,0.780835,9.99726,0.052748,52.220001,49.172989,0.061965,1.174748
4,ASEA,0.34525,9.99726,0.016434,16.0149,14.345758,0.116351,7.079854
5,XT,0.952837,6.909589,0.187078,57.330002,64.15508,-0.106384,-1.990208
6,IXG,0.889357,9.99726,0.079686,82.419998,77.263296,0.066742,0.837565
7,IXJ,0.960792,9.99726,0.110999,82.510002,82.80173,-0.003523,-0.039107
8,KXI,0.961689,9.99726,0.071468,63.099998,60.870746,0.036623,0.512436
9,RXI,0.949133,9.99726,0.119109,162.649994,163.831243,-0.00721,-0.085879


### Construct a portfolio

In [7]:
instruments_select = ['IVV', '3073.HK', 'VPL', 'IEV', 'ASEA']

df_portfolio = df_valuation_metrics[df_valuation_metrics['ticker'].isin(instruments_select)]
df_portfolio = addCashPortfolio(df_portfolio)
df_portfolio['portfolio_allocation'] = allocatePortfolio(df_portfolio['over_value_years'], with_cash=True)

df_portfolio

,ticker,r2_train,train_years,annualized_return,currenct_price,fair_price,over_value_range,over_value_years,portfolio_allocation
0,IVV,0.970875,9.99726,0.134298,439.100006,410.225532,0.070387,0.524109,0.104360
1,3073.HK,0.916889,10.00274,0.082725,51.540001,52.423723,-0.016857,-0.139451,0.301734
2,VPL,0.890417,9.99726,0.067471,75.190002,77.105931,-0.024848,-0.167653,0.315661
3,IEV,0.780835,9.99726,0.052748,52.220001,49.172989,0.061965,1.174748,0.036849
4,ASEA,0.34525,9.99726,0.016434,16.0149,14.345758,0.116351,7.079854,0.000003
5,cash,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.241393
